# Capstone project:
# La Jolla Rental House Prices in relation to Venues
### Author: James Lee

## 1. Background

San Diego is a city on Pacific coast of California with around 1.5  million residents. Because of its warm climate and beaches, San Diego has one of the highest cost of living in California. La Jolla is a seaside neighborhood where UC San Diego is located at and has an even higher cost of living compared to other parts of San Diego. <br> 

The main purpose of this project is to analyze the relationships between the venues and rental prices across different areas near UCSD.
<br> 


## 2. Dataset

* I will be obtaining all the data for locations and venues in San Diego through **Foursquare API**.
<br> <br> 
* The rental prices dataset was obtained from Zillow. The particular dataset does not include raw rental prices. The values are weighted to rental housing stock to ensure representativeness acorss the entire market. The values are caculated so that they fall into the 40-60th percentile rnage for all rents in a given region in order to represent the average rent prices.
<br>
Link address: http://files.zillowstatic.com/research/public_v2/zori/Metro_ZORI_AllHomesPlusMultifamily_SSA.csv  

<br>


1) Import all the necessary libraries

In [112]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Libraries imported.')

Libraries imported.


2) Input the Foursquare API info below

In [113]:
CLIENT_ID = 'O5XRHQWTDAQI4X53O3LXOAQOUYSIJ5UTGL1W21E43AT3PDCT' # your Foursquare ID
CLIENT_SECRET = '5U5LXOZT5B4FJNIMTQCPX5I4GTR4WNAIT2APEDB2NJG3TYV2' # your Foursquare Secret
ACCESS_TOKEN = '2VMIKNFIAIA3E4RFB1EJHDPNRHDJK1VRT3D1IKVULUZV20ZY' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O5XRHQWTDAQI4X53O3LXOAQOUYSIJ5UTGL1W21E43AT3PDCT
CLIENT_SECRET:5U5LXOZT5B4FJNIMTQCPX5I4GTR4WNAIT2APEDB2NJG3TYV2


Get the center latitude and longitude of San Diego

In [114]:
address = 'UCSD'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

32.87935255 -117.2311004938553


Now let's query restaurants

In [115]:
search_query = 'restaurant'
radius = 5000
print(search_query + ' .... OK!')

restaurant .... OK!


In [116]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

In [127]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
df.head()

<ipython-input-127-d9c04fdf16cd>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,58cdfbba110d037d32772e8c,BJ's Restaurant & Brewhouse,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1606639772,False,8785 Villa La Jolla Dr,at Nobel Dr,32.868009,-117.233232,"[{'label': 'display', 'lat': 32.86800867565035...",...,CA,United States,"[8785 Villa La Jolla Dr (at Nobel Dr), San Die...",476703,https://www.grubhub.com/restaurant/bjs-restaur...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,562865916
1,43f66fd0f964a520b32f1fe3,Leucadia Pizzeria & Italian Restaurant,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1606639772,False,7748 Regents Rd Ste 303,at Arriba St,32.862367,-117.224743,"[{'label': 'display', 'lat': 32.86236681380613...",...,CA,United States,"[7748 Regents Rd Ste 303 (at Arriba St), San D...",1729422,https://www.grubhub.com/restaurant/leucadia-pi...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,129967545
2,4b22b29cf964a520074c24e3,Indulge Restaurant & Lounge,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",v-1606639772,False,4550 La Jolla Village Dr,NaN,32.874462,-117.208366,"[{'label': 'display', 'lat': 32.8744618808593,...",...,CA,United States,"[4550 La Jolla Village Dr, San Diego, CA 92122]",1601086,https://www.grubhub.com/restaurant/indulge-res...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
3,4c424971d7fad13ad4f308da,Shores Restaurant,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",v-1606639772,False,8110 Camino Del Oro,NaN,32.855336,-117.258289,"[{'label': 'display', 'lat': 32.85533644046734...",...,CA,United States,"[8110 Camino Del Oro, La Jolla, CA 92037]",1871024,https://www.grubhub.com/restaurant/the-shores-...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,67692138
4,51ec7e68498e77b029c1ecfa,La Jolla Playhouse Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1606639772,False,NaN,NaN,32.871754,-117.240720,"[{'label': 'display', 'lat': 32.871754, 'lng':...",...,CA,United States,"[San Diego, CA]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df['categories'] = df.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df.columns = [column.split('.')[-1] for column in df.columns]

df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,BJ's Restaurant & Brewhouse,Brewery,8785 Villa La Jolla Dr,at Nobel Dr,32.868009,-117.233232,"[{'label': 'display', 'lat': 32.86800867565035...",1278,92037,US,San Diego,CA,United States,"[8785 Villa La Jolla Dr (at Nobel Dr), San Die...",58cdfbba110d037d32772e8c
1,Leucadia Pizzeria & Italian Restaurant,Italian Restaurant,7748 Regents Rd Ste 303,at Arriba St,32.862367,-117.224743,"[{'label': 'display', 'lat': 32.86236681380613...",1982,92122,US,San Diego,CA,United States,"[7748 Regents Rd Ste 303 (at Arriba St), San D...",43f66fd0f964a520b32f1fe3
2,Indulge Restaurant & Lounge,New American Restaurant,4550 La Jolla Village Dr,NaN,32.874462,-117.208366,"[{'label': 'display', 'lat': 32.8744618808593,...",2194,92122,US,San Diego,CA,United States,"[4550 La Jolla Village Dr, San Diego, CA 92122]",4b22b29cf964a520074c24e3
3,Shores Restaurant,New American Restaurant,8110 Camino Del Oro,NaN,32.855336,-117.258289,"[{'label': 'display', 'lat': 32.85533644046734...",3689,92037,US,La Jolla,CA,United States,"[8110 Camino Del Oro, La Jolla, CA 92037]",4c424971d7fad13ad4f308da
4,La Jolla Playhouse Restaurant,Restaurant,NaN,NaN,32.871754,-117.240720,"[{'label': 'display', 'lat': 32.871754, 'lng':...",1234,NaN,US,San Diego,CA,United States,"[San Diego, CA]",51ec7e68498e77b029c1ecfa


Now let's visualize all the restaurants that are nearby

In [129]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df.lat, df.lng, df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map